# Prepping the data

## Scraping data about Canada from wiki

In [68]:
import pandas as pd

link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
Canada_df=pd.read_html(link, header=0)[0]
desired_cols=['Postcode', 'Borough', 'Neighbourhood']

In [69]:
Canada_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Removing all rows with borough that is "not assigned"

In [70]:
Canada_df1=Canada_df[Canada_df.Borough !='Not assigned']
Canada_df1=Canada_df1.reset_index()
Canada_df1=Canada_df1.drop(['index'], axis=1 )

In [71]:
Canada_df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [72]:
#get the index of the neighbourhood that is not assigned
idx=Canada_df1.index[Canada_df1.Neighbourhood.str.contains('Not assigned')]
#assign neighbourhood that is not assigned to the same value as the borough
Canada_df1.loc[idx, 'Neighbourhood' ]=Canada_df1.loc[idx, 'Borough' ]

## Combine rows with the same postalcode

In [73]:
Canada_df2=Canada_df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

## Print out number of rows

In [74]:
Canada_df2.shape

(103, 3)

## add lattitude and longitude

In [75]:
# read in lattitude and longitude from csv file
latlong_df=pd.read_csv('http://cocl.us/Geospatial_data')
latlong_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [76]:
# change column of "postcode" in Canada_df to "Postal Code"

Canada_df2=Canada_df.rename(columns={'Postcode': 'Postal Code'})
Canada_df2.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [77]:
Final_df=pd.merge(Canada_df2, latlong_pd, on='Postal Code')
Final_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
